In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')

### Q1 Solution: Number of records in Jan 2021 FHV data


In [3]:
len(df)

1154112

In [4]:
original_df_len=len(df)

In [5]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [6]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [7]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667


### Q2 Solution. Average duration in Jan 2021 FHV


In [8]:
df.duration.mean()

19.1672240937939

In [9]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [10]:
df_outlier_removed_len=len(df)

In [12]:
drop_len=original_df_len-df_outlier_removed_len

In [13]:
drop_len

44286

### Q3 solution :What's the fractions of missing values for the pickup location ID??

In [14]:
df["PUlocationID"].isnull().sum()/len(df)*100

83.52732770722618

In [14]:
df["PUlocationID"].fillna(-1, inplace=True)
df["DOlocationID"].fillna(-1, inplace=True)

In [15]:
df.head(5)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000


In [19]:
categorical = ['PUlocationID', 'DOlocationID']
df[categorical] = df[categorical].astype(str)

In [20]:
train_dicts = df[categorical].to_dict(orient='records')

In [21]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

### Q4 Solution: Dimensionality after OHE 


In [27]:
X_train.shape

(1109826, 525)

### Q5 solution: RMSE on train


In [29]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.5285191072072

### Q6 Solution: RMSE on validation


In [32]:
df_val = pd.read_parquet('./data/fhv_tripdata_2021-02.parquet')
df_val['duration'] = df_val.dropOff_datetime - df.pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

In [33]:
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [34]:
df_val["PUlocationID"].fillna(-1, inplace=True)
df_val["DOlocationID"].fillna(-1, inplace=True)

In [35]:
categorical = ['PUlocationID', 'DOlocationID']
df_val[categorical] = df_val[categorical].astype(str)

In [36]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [37]:
X_val = dv.transform(val_dicts)

In [38]:
y_val = df_val[target].values

In [39]:
y_pred = lr.predict(X_val)

In [40]:
mean_squared_error(y_val, y_pred, squared=False)

11.014283163400654